In [1]:
"""
Conductance Quantization Analysis - Measuring G0
=================================================
Measures G0 = 2e^2/h from gold wire break-junction PicoScope data.

Method: Scan trial G0 values and find which one maximizes
integer clustering in the conductance histogram.

Usage: Edit CONFIG section, then run.
"""
import csv, os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ============ CONFIG ============
R_D1=10230.0; R_D2=99.3; R_A=327500.0; V_SUPPLY=4.96
dR_D1=112.3; dR_D2=1.293; dR_A=3375.0; dV_SUPPLY=0.0497
DATA_DIR=Path("."); FILE_PREFIX="working_traces/trace_"; OUTPUT_DIR=Path("./output2")
RAIL_THRESHOLD=-8.5; OPEN_MARGIN=0.2; MIN_STEP_POINTS=15; CONTEXT_POINTS=30
G0_SCAN_MIN=4e-5; G0_SCAN_MAX=12e-5; G0_SCAN_POINTS=2000
# ================================

In [2]:

V_G=V_SUPPLY*R_D2/(R_D1+R_D2); TRANSIMPEDANCE=R_A*V_G

In [3]:
def load_trace(fp):
    t,v=[],[]
    with open(fp,'r',encoding='utf-8-sig') as f:
        r=csv.reader(f); next(r); next(r); next(r)
        for row in r:
            if len(row)>=2:
                try: tt=float(row[0])
                except: continue
                try: vv=float(row[1])
                except: vv=np.nan
                t.append(tt); v.append(vv)
    return np.array(t),np.array(v)

def find_step_regions(t,v,v_open):
    vs=np.where(np.isnan(v),-100,v)
    m=(vs<v_open-OPEN_MARGIN)&(vs>RAIL_THRESHOLD)
    regions=[]; inr=False; s=0
    for i in range(len(m)):
        if m[i] and not inr: s=i; inr=True
        elif not m[i] and inr:
            if i-s>3: regions.append((max(0,s-CONTEXT_POINTS),min(len(t),i+CONTEXT_POINTS)))
            inr=False
    if inr and len(t)-s>3: regions.append((max(0,s-CONTEXT_POINTS),len(t)))
    merged=[]
    for r in regions:
        if merged and r[0]<=merged[-1][1]: merged[-1]=(merged[-1][0],max(merged[-1][1],r[1]))
        else: merged.append(r)
    good=[]
    for s,e in merged:
        vr=v[s:e]; vv=vr[~np.isnan(vr)]
        if np.sum((vv>RAIL_THRESHOLD)&(vv<v_open-OPEN_MARGIN))>=MIN_STEP_POINTS: good.append((s,e))
    return good

def integer_score(G_raw,g0):
    g=G_raw/g0; g=g[(g>0.3)&(g<7)]
    if len(g)<10: return 0.
    h,e=np.histogram(g,bins=np.linspace(0.3,7,200))
    c=(e[:-1]+e[1:])/2; hs=gaussian_filter1d(h.astype(float),sigma=2)
    return sum(np.sum(hs[(c>n-0.15)&(c<n+0.15)]) for n in range(1,7))

In [4]:
def main():
    OUTPUT_DIR.mkdir(exist_ok=True)
    files=sorted(DATA_DIR.glob(f"{FILE_PREFIX}*.csv"))
    if not files: print("No files found"); return
    print(f"Found {len(files)} files, V_G={V_G*1000:.2f} mV, Transimpedance={TRANSIMPEDANCE:.1f} V/S")
    traces=[(load_trace(f),f.stem.replace(FILE_PREFIX,'#')) for f in files]
    hi=[]; 
    for (t,v),_ in traces: hi.extend(v[~np.isnan(v)&(v>2)].tolist())
    V_OPEN=np.median(hi) if hi else 2.94
    print(f"V_open={V_OPEN:.3f} V")
    all_sv=[]; all_sr=[]
    for (t,v),lab in traces:
        for s,e in find_step_regions(t,v,V_OPEN):
            all_sr.append((t[s:e],v[s:e],lab))
            vv=v[s:e]; vv=vv[~np.isnan(vv)]; sv=vv[(vv>RAIL_THRESHOLD)&(vv<V_OPEN-OPEN_MARGIN)]
            all_sv.extend(sv.tolist())
    all_sv=np.array(all_sv)
    print(f"Step regions: {len(all_sr)}, data points: {len(all_sv)}")
    G_raw=(V_OPEN-all_sv)/TRANSIMPEDANCE
    print("Scanning G0...")
    g0t=np.linspace(G0_SCAN_MIN,G0_SCAN_MAX,G0_SCAN_POINTS)
    sc=np.array([integer_score(G_raw,g) for g in g0t])
    G0c=g0t[np.argmax(sc)]
    g0f=np.linspace(G0c-1e-5,G0c+1e-5,3000)
    scf=np.array([integer_score(G_raw,g) for g in g0f])
    G0m=g0f[np.argmax(scf)]; DVm=G0m*TRANSIMPEDANCE
    G0th=2*(1.602176634e-19)**2/6.62607015e-34
    hm=scf.max()*0.5; ab=g0f[scf>hm]
    err_s=(ab[-1]-ab[0])/2.236 if len(ab)>1 else G0m*0.05
    ds=np.sqrt(dR_D1**2+dR_D2**2)
    rc=np.sqrt((dR_A/R_A)**2+(dV_SUPPLY/V_SUPPLY)**2+(dR_D2/R_D2)**2+(ds/(R_D1+R_D2))**2)
    err_c=G0m*rc; err_t=np.sqrt(err_s**2+err_c**2)
    print(f"\n{'='*50}\nRESULT\n{'='*50}")
    print(f"G0 = ({G0m*1e5:.3f} +/- {err_t*1e5:.3f}) x 10^-5 S")
    print(f"Theory = {G0th*1e5:.3f} x 10^-5 S")
    print(f"Error = {(G0m/G0th-1)*100:+.1f}%")
    # Figures
    fig=plt.figure(figsize=(14,16))
    ax1=fig.add_subplot(4,1,1); ax1.plot(g0t*1e5,sc,'b-')
    ax1.axvline(G0m*1e5,color='r',ls='--',label=f'Measured={G0m*1e5:.3f}')
    ax1.axvline(G0th*1e5,color='g',ls=':',label=f'Theory={G0th*1e5:.3f}')
    ax1.set_xlabel('Trial G0 (x1e-5 S)'); ax1.set_ylabel('Score'); ax1.legend(); ax1.grid(True,alpha=0.2)
    ax1.set_title('G0 scan: maximize integer clustering')
    ax2=fig.add_subplot(4,1,2); b2=np.linspace(-6,V_OPEN+0.2,500)
    h2,e2=np.histogram(all_sv,bins=b2); c2=(e2[:-1]+e2[1:])/2
    ax2.bar(c2,h2,width=c2[1]-c2[0],color='steelblue',alpha=0.7,edgecolor='none')
    for n in range(1,7):
        ve=V_OPEN-n*DVm
        if ve>-6: ax2.axvline(ve,color='r',ls='--',alpha=0.6); ax2.text(ve,max(h2)*0.8,f'n={n}',ha='center',color='r')
    ax2.set_xlabel('V_out (V)'); ax2.set_ylabel('Counts'); ax2.grid(True,alpha=0.2)
    ax2.set_title(f'Voltage histogram (DV={DVm:.3f} V)')
    ax3=fig.add_subplot(4,1,3); gp=G_raw/G0m; gp=gp[(gp>0)&(gp<8)]
    ax3.hist(gp,bins=150,range=(0,8),color='steelblue',edgecolor='none',alpha=0.8)
    for n in range(1,8): ax3.axvline(n,color='r',ls='--',alpha=0.5)
    ax3.set_xlabel('Conductance (G0_meas)'); ax3.set_ylabel('Counts'); ax3.grid(True,alpha=0.2)
    ax3.set_title(f'Conductance histogram (G0={G0m*1e5:.3f}e-5 S)')
    ax4=fig.add_subplot(4,1,4)
    ax4.errorbar([G0m*1e5],[1],xerr=[err_t*1e5],fmt='ro',ms=12,capsize=8,capthick=2,lw=2,
                 label=f'Measured: ({G0m*1e5:.3f}+/-{err_t*1e5:.3f})e-5 S')
    ax4.axvline(G0th*1e5,color='g',lw=2,ls='--',label=f'Theory: {G0th*1e5:.3f}e-5 S')
    ax4.set_xlabel('G0 (x1e-5 S)'); ax4.set_yticks([]); ax4.legend(loc='upper left')
    ax4.grid(True,alpha=0.2,axis='x'); ax4.set_xlim(5,10); ax4.set_title('Measured vs Theory')
    fig.tight_layout(); fig.savefig(OUTPUT_DIR/'fig3_G0_measurement.png',dpi=150); plt.close()
    print(f"\nSaved to {OUTPUT_DIR.resolve()}")

if __name__=='__main__': main()

Found 39 files, V_G=47.68 mV, Transimpedance=15616.1 V/S
V_open=2.941 V
Step regions: 223, data points: 116711
Scanning G0...

RESULT
G0 = (7.972 +/- 0.681) x 10^-5 S
Theory = 7.748 x 10^-5 S
Error = +2.9%

Saved to /Users/zhangyongqi/Desktop/PHY4400/Lab1/data/output2
